In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    get_scheduler,
    default_data_collator,
    SchedulerType
)
import os
import json
from itertools import chain
from datasets import load_dataset

In [3]:
train_file = 'combine-mistral.jsonl'
tokenizer = AutoTokenizer.from_pretrained(
    'mistralai/Mistral-7B-v0.1',
)
tokenizer.add_bos_token = False
tokenizer.add_eos_token = False
text_column_name = 'text'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
raw_datasets = load_dataset(
    'json',
    data_files=train_file,
    split='train'
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

In [6]:
filename = os.path.split(train_file)[1]
column_names = raw_datasets.column_names
tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=column_names,
    load_from_cache_file=True,
    cache_file_name=f'./{filename}-tokenized',
    num_proc=20,
)

tokenized_datasets

Map (num_proc=20):   0%|          | 0/37588007 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 37588007
})

In [7]:
block_size = 32768

def group_texts(examples):
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i: i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    load_from_cache_file=True,
    cache_file_name=f'./{filename}-grouped-{block_size}',
    num_proc=20,
)

Map (num_proc=20):   0%|          | 0/37588007 [00:00<?, ? examples/s]

In [9]:
len(lm_datasets) * block_size

9984573440